Import Statments and Parameters:

--------------------------------------------------------------------

IMPORTANT: Run these pip installs,

If using miniconda: ```conda create --name project_env python=3.11```

```pip install tensorflow keras numpy pandas requests Pillow scikit-learn matplotlib opencv-python jupyter ipykernel```

Download dataset here: https://iris.di.ubi.pt/ubipr.html (Original version, Change DATASET_PATH accordingly)

--------------------------------------------------------------------

`IMG_SIZE` - All Images scaled to this size.

`BATCH_SIZE` - Groups pairs of images for siamese training.

In [3]:
from lib import *

# -----------------------------
# PARAMETERS
# -----------------------------
IMG_SIZE = (105, 105)
BATCH_SIZE = 32
EPOCHS = 4  # increase for better results
STEPS_PER_EPOCH = 50
# -----------------------------

env = load_env("paths.env")

if env is not None:
    DATASET_PATH, SAVE_PATH, GALLERY_PATH, QUERY_IMAGE_PATH = build_paths(env)

print("DATASET_PATH:", DATASET_PATH)
print("SAVE_PATH:", SAVE_PATH)
print("GALLERY_PATH:", GALLERY_PATH)
print("QUERY_IMAGE_PATH:", QUERY_IMAGE_PATH)


DATASET_PATH: C:\Users\Gunnar\Files\School_Stuff\IntroToAi\AiFinalProject\UBIPeriocular
SAVE_PATH: C:\Users\Gunnar\Files\School_Stuff\IntroToAi\AiFinalProject\siamese_eye_model.keras
GALLERY_PATH: C:\Users\Gunnar\Files\School_Stuff\IntroToAi\AiFinalProject\Gallery
QUERY_IMAGE_PATH: C:\Users\Gunnar\Files\School_Stuff\IntroToAi\AiFinalProject\query_image.jpg


Image Preparation: Generates tensorflow dataset for training.

In [5]:
# -----------------------------
# IMAGE PREPARATION
# -----------------------------

images_dict = load_images_by_filename(DATASET_PATH, IMG_SIZE)
for person, imgs in images_dict.items():
    print(person, len(imgs))  # sanity check

train_ds = make_tf_dataset(images_dict, BATCH_SIZE, IMG_SIZE)

C100_S1 15
C100_S2 15
C101_S1 15
C101_S2 14
C102_S1 15
C102_S2 15
C103_S1 15
C103_S2 15
C104_S1 15
C104_S2 15
C105_S1 15
C105_S2 15
C106_S1 15
C106_S2 15
C107_S1 15
C107_S2 15
C108_S1 15
C108_S2 15
C109_S1 15
C109_S2 15
C10_S1 15
C10_S2 15
C110_S1 15
C110_S2 15
C111_S1 15
C111_S2 15
C112_S1 15
C112_S2 15
C113_S1 15
C114_S1 15
C115_S1 15
C116_S1 15
C117_S1 15
C118_S1 15
C119_S1 15
C11_S1 15
C120_S1 15
C121_S1 15
C121_S2 15
C122_S1 15
C122_S2 15
C123_S1 15
C124_S1 15
C125_S1 15
C125_S2 15
C126_S1 15
C126_S2 15
C127_S1 15
C127_S2 15
C128_S1 15
C128_S2 15
C129_S1 15
C12_S1 15
C130_S1 15
C131_S1 15
C132_S1 15
C133_S1 15
C134_S1 15
C135_S1 15
C136_S1 15
C137_S1 15
C138_S1 15
C139_S1 15
C13_S1 15
C140_S1 15
C141_S1 15
C142_S1 15
C143_S1 15
C144_S1 15
C145_S1 15
C146_S1 15
C147_S1 15
C147_S2 15
C148_S1 15
C148_S2 15
C149_S1 15
C149_S2 15
C14_S1 15
C150_S1 15
C150_S2 15
C151_S1 15
C151_S2 15
C152_S1 15
C152_S2 15
C153_S1 15
C154_S1 15
C155_S1 15
C156_S1 15
C157_S1 15
C158_S1 15
C159_S1 15
C15_S

Training:

In [6]:
# -----------------------------
# TRAINING
# -----------------------------
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

early_stop = EarlyStopping(
    monitor='loss',        # or 'val_loss' if using validation
    patience=5,            # stop after 5 epochs with no improvement
    restore_best_weights=True
)

model = create_siamese_model(input_shape=(*IMG_SIZE, 3))
model.compile(loss=contrastive_loss,
              optimizer=Adam(learning_rate=1e-4),
              metrics=[siamese_accuracy])
model.summary()

model.fit(
    train_ds,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 4096)      │ 38,960,448 │ input_layer_1[0]… │
│ (Functional)        │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1)         │          0 │ functional[0][0], │
│                     │                   │            │ functional[1][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,960,448 (148.62 MB)

 Trainable params: 38,960,448 (148.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/4
50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 592ms/step - loss: 0.1712 - siamese_accuracy: 0.7656
Epoch 2/4
50/50 ━━━━━━━━━━━━━━━━━━━━ 37s 736ms/step - loss: 0.1184 - siamese_accuracy: 0.8781
Epoch 3/4
50/50 ━━━━━━━━━━━━━━━━━━━━ 38s 765ms/step - loss: 0.1050 - siamese_accuracy: 0.8825
Epoch 4/4
50/50 ━━━━━━━━━━━━━━━━━━━━ 39s 786ms/step - loss: 0.0902 - siamese_accuracy: 0.9044


In [7]:
# Call this after training:
save_siamese_model(model, str(SAVE_PATH))  # Or a folder name

Model saved to: C:\Users\Gunnar\Files\School_Stuff\IntroToAi\AiFinalProject\siamese_eye_model.keras


Compare Eyes: Uses identify_eye() to compare query_image with images in gallery and outputs highest similarity image.

------------------------------------------------------------------
IMPORTANT:

Run previous definition cells.

Gallery should contain subfolders for each person containing images of their eyes.

Subfolder name should be persons names/identifier.

------------------------------------------------------------------

In [8]:
model = load_siamese_model(str(SAVE_PATH))

gallery_dict = load_gallery_embeddings(GALLERY_PATH)

identity, confidence = identify_eye(model, QUERY_IMAGE_PATH, gallery_dict, IMG_SIZE, margin=1.0, threshold=80.0)

print(f"Identified as: {identity} with confidence {confidence:.2f}%")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Gunnar\\Files\\School_Stuff\\IntroToAi\\AiFinalProject\\Gallery'